# Text Generation - DevFest Nantes 2021 - IArt

 
<br>  
Brique n°2 de la conférence **IArt ou comment apprendre à une machine à tagger et à composer**
 
<br>  

<img src="images/image1.jpg" alt="Text Generation" width="400" />

### Objectifs
 
<br>  
- Introduction à la génération de texte
<br>  
<br>  
- Générer les paroles de notre tube 📜📝📋

### Etapes

<br>  
1. Chargement des données
<br>  
    - Chargement du dataset dans une seule "string" et suppression de certains caractères en trop.
    - On va ensuite tokeniser cette chaine de caractère (i.e. transformation en séquence d'identifiants).  
<br>  
2. Préparation du dataset pour la modélisation
<br>  
    - On va séparer notre jeu de données en séquences de X caractères
    - Pour chaque séquence, on sépare en input / output:
        - Input : La séquence moins le dernier caractère
        - Output : La prochaine séquence à prédire, i.e. la séquence moins le premier caractère
    - On termine par créer un générateur de batchs, qui sera utilisé pour l'apprentissage de notre modèle  
<br>  
3. Modélisation
<br>  
    - On va définir un modèle de type RNN adapté à notre problème.  
<br>  
4. Entrainement
<br>  
    - On entraine le modèle sur l'intégralité de nos données.
        - Pas obligé d'avoir un jeu de validation, on veut volontairement overfitter le modèle pour apprendre au mieux le "genre" des paroles en entrée.  
<br>  
5. Prédictions
<br>  
    - On termine par générer de nouvelles paroles à partir d'un texte en entrée.
        - La fonction de génération inclut une sélection "aléatoire" selon la distribution des prédictions obtenues par le modèle, pour apporter un peu de variété.  
<br>  

---

### Données

<br>  
On va utiliser un ensemble de paroles de musique de Hip-Hop. Cet ensemble provient d'un scrapping de Metro Lyrics, mais il est devenu un peu compliqué de trouver sur internet. On a pu le récupérer ici : https://github.com/ludovicaschaerf/TMCI_Project/blob/master/data/380000-lyrics-from-metrolyrics.zip

Ce jeu de données à été préprocessé avant d'être poussé sur ce Git. On a filtré les musiques de type "Hip-Hop" et réalisé quelques prétraitements sur les paroles. Ces derniers ne sont pas parfaits, mais ça fait largement l'affaire pour notre projet.  

Le code utilisé est mis à disposition à la fin de ce notebook pour information.  


(Certaines chansons ne sont pas forcément en anglais, mais il y en a très peu, donc on ignore le problème)

---

### Imports

In [1]:
# Divers
import os
import re
import time
import random
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import display, HTML, Javascript, clear_output

# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Center figures
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

---

### Fonctions utilitaires

<br>  
On commence par définir certaines fonctions utilitaires :

- **hide_toggle** : permet de "masquer" certaines cellules de ce notebook, ce qui permet d'apporter un peu plus de clarté ;)   

In [3]:
def hide_toggle(for_next=False, text_display='Toggle show/hide'):
    '''Function to hide a notebook cell'''
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    target_cell = this_cell  # target cell to control with toggle

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = f"""
        <script>
            function {js_f_name}() {{
                {target_cell}.find('div.input').toggle();
            }}

        </script>

        <a href="javascript:{js_f_name}()" id="{js_f_name}">{text_display}</a>
    """
    
    js = f'''
            var output_area = this;
            var cell_element = output_area.element.parents('.cell');
            var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
            var current_cell = Jupyter.notebook.get_cell(cell_idx);
            $(current_cell.element[0]).find('div.input').toggle();
            Jupyter.notebook.select(cell_idx +  1);
            Jupyter.notebook.focus_cell();
         '''

    display(HTML(html))
    display(Javascript(js))

hide_toggle(text_display='Toggle show/hide --- fonction hide_toggle')

<IPython.core.display.Javascript object>

---

### 1. Chargement des données

<br>

- Chargement du dataset dans une seule "string" et suppression de certains caractères en trop.
- On va ensuite tokeniser cette chaine de caractère (i.e. transformation en séquence d'identifiants).
<br>   
<br>   

On définit quelques fonctions pour clarifier le notebook :

- **text_from_ids** : Fonction pour retransformer une séquence d'identifiants en texte  

In [4]:
def text_from_ids(seq: list, ids_from_chars):
    '''Fonction pour retransformer une séquence d'identifiants en texte
    
    Args:
        seq (list): liste d'identifiants à transformer en texte
        ids_from_chars: tokenizer utilisé pour créer les identifiants
    Returns:
        str: texte en sortie
    '''
    # Utilitaire TF pour transformer les IDs en texte
    chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
    
    # Apply
    text = tf.strings.reduce_join(chars_from_ids(seq), axis=-1).numpy().decode('utf-8')

    # Return
    return text

hide_toggle(text_display='Toggle show/hide --- fonction text_from_ids')

<IPython.core.display.Javascript object>

In [5]:
# Chargement du jeu de données et concaténation dans une seule string
full_lyrics = " . ".join(pd.read_csv("lyrics/lyrics_hip_hop.csv", sep=';', encoding='utf-8')["lyrics"].values)
# On pourrait tout mettre en lower pour diminuer le nombre de "classes" possible (i.e. de caractère possible)


# Attention, l'entrainement d'un modèle sur l'intégralité des données avec une GPU GTX 1070 peut prendre plus de deux heures
# Vous pouvez donc réduire la taille du dataset si vous le souhaitez
# e.g. full_lyrics = full_lyrics[:1000000] 

In [6]:
# On supprime certains caractères en trop (on aurait pu le faire dans le preprocessing)
full_lyrics = re.sub(r"[&#£$€%<>=@\^`\{\}|~\t\x18\x7f]", '', full_lyrics)

In [7]:
# Analyse de notre vocabulaire -> 1 caractère = 1 classe de notre modèle
vocab = sorted(set(full_lyrics))
print(f"Notre jeu de données est composé de {len(full_lyrics)} caractères.")
print(f"Il y a {len(vocab)} caractères uniques.")

Notre jeu de données est composé de 59100439 caractères.
Il y a 67 caractères uniques.


In [8]:
# Tokenizer
ids_from_chars = preprocessing.StringLookup(vocabulary=vocab, mask_token=None)
# Application sur nos données
lyrics_ids = ids_from_chars(tf.strings.unicode_split(full_lyrics, 'UTF-8'))
# Création d'un Dataset au format tensorflow
lyrics_ids_dataset = tf.data.Dataset.from_tensor_slices(lyrics_ids)

---

### 2. Préparation du dataset pour la modélisation

<br>

- On va séparer notre jeu de données en séquences de X caractères
- Pour chaque séquence, on sépare en input / output:
    - Input : La séquence moins le dernier caractère
    - Output : La prochaine séquence à prédire, i.e. la séquence moins le premier caractère
- On termine par créer un générateur de batchs, qui sera utilisé pour l'apprentissage de notre modèle
<br>   
<br>   

On définit quelques fonctions pour clarifier le notebook :

- **split_input_target** : Fonction pour séparer une séquence (suite de caractères) en input/output 

In [9]:
def split_input_target(sequence):
    '''Fonction pour séparer une séquence (suite de caractères) en input/output
    
    Args:
        sequence: suite de caractères
    Returns:
        ? : input
        ? : output
    '''
    input_text = sequence[:-1]  # Input : toute la séquence sauf le dernier caractère
    target_text = sequence[1:]  # Output : toute la séquence sauf le premier caractère
    # Returns
    return input_text, target_text

hide_toggle(text_display='Toggle show/hide --- fonction split_input_target')

<IPython.core.display.Javascript object>

In [10]:
# Paramètres
sequence_length = 100  # Longueur des séquences
batch_size = 1024  # Taille des batchs pour l'apprentissage, utile ici pour le generator
buffer_size = 10000  # Utilisé pour mélanger le dataset sans tout charger en mémoire

In [11]:
# Création des séquences (length + 1 pour ajouter le prochain caractère)
sequences = lyrics_ids_dataset.batch(sequence_length + 1, drop_remainder=True)
# Gestion des inputs/outputs
model_dataset = sequences.map(split_input_target)

In [12]:
# Affichage - exemple d'un couple input/output
for input_example, target_example in model_dataset.take(1):
    print("Input :", text_from_ids(input_example, ids_from_chars))
    print("Target:", text_from_ids(target_example, ids_from_chars))

Input : . Timbo When you hit me on my phone betta know what cha want when you call me you already know on th
Target:  Timbo When you hit me on my phone betta know what cha want when you call me you already know on the


In [13]:
# Dataset (generator) final à utiliser en entrée du modèle
model_dataset = model_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

---

### 3. Modélisation
<br>

- On va définir un modèle de type RNN adapté à notre problème
<br>   
<br>   

On définit quelques fonctions & classes pour clarifier le notebook :

- **ModelGenerationText** : Modèle principal pour la génération de texte
- **get_model_loss** : Fonction pour définir une loss à notre modèle

In [14]:
class ModelGenerationText(tf.keras.Model):
    '''Modèle principal pour la génération de texte'''
    
    def __init__(self, vocab_size: int, embedding_dim: int, rnn_units: int):
        '''Initialisation de la classe
        
        Args:
            vocab_size (int): taille du vocabulaire (i.e. classes possibles)
            embedding_size (int): Dimension de l'embedding
            rnn_units (int): Nombre d'unités RNN
        '''
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, trainable=True)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training: bool = False):
        '''Appel au model
        
        Args:
            inputs: données en entrée
            states: états RNN à utiliser si précisé
            return_state: si les états RNN doivent être retournés
            training (bool): si on est en phase d'entrainement
        Returns:
            ?: outputs du modèle
            ?: (optionnel) états des RNNs
        '''
        x = self.embedding(inputs, training=training)
        # Si pas de states de précisé, on initialise les RNNs
        if states is None:
            states = self.gru.get_initial_state(x)
        # Récupération de la sortie des RNNs ainsi que des states
        x, states = self.gru(x, initial_state=states, training=training)
        # Suite du modèle
        x = self.dense(x, training=training)

        # Return en fonction de l'argument return_state
        if return_state:
            return x, states
        else:
            return x

hide_toggle(text_display='Toggle show/hide --- classe ModelGenerationText')        

<IPython.core.display.Javascript object>

In [15]:
def get_model_loss():
    '''Fonction pour définir une loss à notre modèle'''
    # From logits car on n'a pas de couche de softmax en sortie, on récupère directement les logits
    # Sparse car on a beaucoup de catégories
    return tf.losses.SparseCategoricalCrossentropy(from_logits=True)

hide_toggle(text_display='Toggle show/hide --- fonction get_model_loss')        

<IPython.core.display.Javascript object>

In [16]:
# Paramètres
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 100  # Dimension de l'embedding
rnn_units = 1024  # Nombre d'unités RNN

In [17]:
# Init. modèle
model = ModelGenerationText(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units)


<img src="images/image2.png" alt="Text Generation" width="600" />

*Image from https://www.tensorflow.org/text/tutorials/text_generation*

In [18]:
# Récupération de la loss & compile de notre modèle
lr = 0.001  # Paramétrable
loss = get_model_loss()
optimizer = tf.keras.optimizers.Adam(lr=lr)
model.compile(optimizer=optimizer, loss=loss)

In [19]:
# On passe le modèle sur 1 séquence pour lui donner les informations de "shape" et pouvoir afficher sa description
for input_example_batch, target_example_batch in model_dataset.take(1):
    model(input_example_batch)
# Affichage
model.summary()

Model: "model_generation_text"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6800      
_________________________________________________________________
gru (GRU)                    multiple                  3459072   
_________________________________________________________________
dense (Dense)                multiple                  69700     
Total params: 3,535,572
Trainable params: 3,535,572
Non-trainable params: 0
_________________________________________________________________


---

### 4. Entrainement
<br>

- On entraine le modèle sur l'intégralité de nos données.
    - Pas obligé d'avoir un jeu de validation, on veut volontairement overfitter le modèle pour apprendre au mieux le "genre" des paroles en entrée.

In [20]:
# On va créer un dossier où sauvegarder nos résultats
subfolder_name = datetime.now().strftime(f"experimentation_%Y_%m_%d-%H_%M_%S")
os.makedirs(subfolder_name)
subfolder_path = os.path.abspath(subfolder_name)
print(f"L'ensemble des résultats seront sauvegardées dans le répertoire {subfolder_path}")

L'ensemble des résultats seront sauvegardées dans le répertoire C:\Users\Alexandre\Dev\Valeuriad\devfest-2021\02_text-generation\experimentation_2021_10_17-15_46_58


In [21]:
# Gestion des sauvegardes du meilleur modèle
# Nom du fichier où sauvegarder le meilleur modèle
checkpoint_filepath = os.path.join(subfolder_path, "best_weights")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
)

In [331]:
# Entrainement
nb_epochs = 20
history = model.fit(model_dataset, epochs=nb_epochs, callbacks=[checkpoint_callback])

Epoch 1/20
96/96 [==============================] - 50s 525ms/step - loss: 3.1832
Epoch 2/20
96/96 [==============================] - 51s 528ms/step - loss: 2.3791
Epoch 3/20
96/96 [==============================] - 51s 527ms/step - loss: 2.1670
Epoch 4/20
96/96 [==============================] - 51s 528ms/step - loss: 1.9914
Epoch 5/20
96/96 [==============================] - 53s 553ms/step - loss: 1.8587
Epoch 6/20
96/96 [==============================] - 51s 529ms/step - loss: 1.7583
Epoch 7/20
96/96 [==============================] - 51s 528ms/step - loss: 1.6793
Epoch 8/20
96/96 [==============================] - 51s 528ms/step - loss: 1.6137
Epoch 9/20
96/96 [==============================] - 51s 528ms/step - loss: 1.5631
Epoch 10/20
96/96 [==============================] - 51s 527ms/step - loss: 1.5231
Epoch 11/20
96/96 [==============================] - 51s 529ms/step - loss: 1.4892
Epoch 12/20
96/96 [==============================] - 51s 528ms/step - loss: 1.4608
Epoch 13/20
9

In [332]:
# On recharge le meilleur modèle
model.load_weights(checkpoint_filepath)

---

### 5. Prédictions
<br>

- On termine par générer de nouvelles paroles à partir d'un texte en entrée.
    - La fonction de génération inclut une sélection "aléatoire" selon la distribution des prédictions obtenues par le modèle, pour apporter un peu de variété.  
<br>   
<br>   

On définit quelques fonctions & classes pour clarifier le notebook :

- **TextPredictor** : Classe qui permet de générer un nouveau caractère à la suite d'une séquence à partir d'un modèle entrainé
- **generate_text** : Fonction pour générer du texte

In [24]:
class TextPredictor(tf.keras.Model):
    '''Classe qui permet de générer un nouveau caractère à la suite d'une séquence à partir d'un modèle entrainé'''
    
    def __init__(self, model, ids_from_chars, use_random_distribution: bool = True, temperature: float = 1.0):
        '''Initialisation de la classe
        
        Args:
            model: modèle à utiliser pour les prédictions
            ids_from_chars: tokenizer   
            use_random_distribution (bool): si le prochain caractère est choisie aléatoirement selon la distribution des prédictions
                ou si on prend systématiquement le plus probable (attention, grande chance de répétition dans le texte de sortie)
            temperature (float): paramètre qui influence sur le choix "aléatoire" du prochain caractère
                < 1.0 -> on va favoriser le caractère le plus probable
                > 1.0 -> on va donner plus de "chances" aux autres caractères
        '''
        super().__init__()
        self.model = model
        self.ids_from_chars = ids_from_chars
        self.use_random_distribution = use_random_distribution
        self.temperature = temperature
        self.chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

        # On définit un mask pour éviter les predictions "[UNK]" (classe 'unknown' du tokenizer)
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float('inf')] * len(skip_ids), # -inf à chaque mauvais index
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        '''Fonction pour générer un nouveau caractère
        
        Args:
            inputs: séquences textuelles en entrée
            states: états des RNNs (défault None)
        '''
        # On commence par transformer les phrases en IDs
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        

        # On fait tourner le modèle sur nos inputs
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        
        # On récupère uniquement la prédiction sur le dernier caractère
        predicted_logits = predicted_logits[:, -1, :]
        if self.use_random_distribution:
            predicted_logits = predicted_logits / self.temperature

        # On applique le masque pour enlever le caractère [UNK]
        predicted_logits = predicted_logits + self.prediction_mask

        # On récupère les IDs à partir des logits obtenues via les prédictions
        if self.use_random_distribution:
            # On récupère un caractère "aléatoire" selon la distribution obtenue
            predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
            predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        else:
            # On prend la caractère le plus probable
            predicted_ids = tf.math.argmax(predicted_logits, axis=-1)

        # On retransforme en texte
        predicted_chars = self.chars_from_ids(predicted_ids)

        # On retourne les caractères obtenus et les états RNN
        return predicted_chars, states

    
hide_toggle(text_display='Toggle show/hide --- classe TextPredictor')

<IPython.core.display.Javascript object>

In [25]:
def generate_text(predictor, original_text: str, nb_chars: int = 100):
    '''Fonction pour générer du texte
    
    Args:
        original_text (str): phrase en entrée
        nb_chars (int): nombre de caractères à créer
    Returns:
        str: new song !
    '''
    start = time.time()

    # Init. variables
    states = None  # Pas d'état au début
    next_char = tf.constant([original_text])
    result = [next_char]

    # Prédiction des nouveaux caractères
    for n in range(nb_chars):
        next_char, states = predictor.generate_one_step(next_char, states=states)
        result.append(next_char)

    # Resultats finaux
    result = tf.strings.join(result)
    
    # Result as string
    result_str = result[0].numpy().decode('utf-8')
    
    # On recolle les points
    result_str = re.sub(r"(?<=\w)\s+\!", '!', result_str)
    result_str = re.sub(r"(?<=\w)\s+\?", '?', result_str)
    result_str = re.sub(r"(?<=\w)\s+\.", '.', result_str)
    
    # Affichage temps
    end = time.time()
    print(f"Temps de traitement : {end - start}s")
    
    return result_str


hide_toggle(text_display='Toggle show/hide --- fonction generate_text')

<IPython.core.display.Javascript object>

In [506]:
# On créé notre predictor
# Le choix de la valeur temperature va généralement dépendre de notre modèle
# Plus le modèle est performant, plus on va souhaité augmenter cette valeur pour avoir un peu d'aléatoire dans les générations
# Sinon, on va avoir tendance à baisser cette valeur pour avoir plus de cohérence dans le texte généré
predictor = TextPredictor(model, ids_from_chars, use_random_distribution=True, temperature=0.5)

In [507]:
# Generation d'un caractère - exemple
predictor.generate_one_step(tf.constant(['yo']))[0].numpy()

array([b'u'], dtype=object)

In [511]:
# On génère de nouvelles paroles

# DISCLAIMER : les paroles de Hip-Hop peuvent contenir des insultes, obscénités, etc..
#              il est donc normal que notre algorithme fasse pareil ...

initial_text = "Devfest is"  # Texte de base à utiliser
new_song_lyrics = generate_text(predictor, initial_text, nb_chars=1000)
# Affichage
print(new_song_lyrics)
# Sauvegarde
save_path = os.path.join(subfolder_path, datetime.now().strftime(f"lyrics_%Y_%m_%d-%H_%M_%S.txt"))
with open(save_path, 'w') as f:
    f.write(new_song_lyrics)

Temps de traitement : 2.4155220985412598s
Devfest is what club is a man but I don't like me I need you I don't see you with the weekend How I promise you broke you don't be blastin' I got the pain I don't care if I can see it in my life I love her a day I got something for this Walk that work it out see my shit bang Now a nigga with a fantass on my back off your back get your head like the barr to the car And the streets want to fuck with the good on the town You better be ready for the niggaz in the pen to the beat They say that you wanna fuck your face I had a lot of thing she love me say I make it a real nigga stay gettin' money I had to tell you when you get a chick shit She can do is come on and get you out here tryna function I know you make my day I got rich I don't wanna be with you I know I'm a soldier I can't do it for my life I got the system you know I'm the best I love to see the life is the way I like I was born in the game I can't stand it I'm a motherfucking game you kn

---
### Misc.

Pour information, le code utilisé pour faire le prétraitement du jeu de données original est le suivant :

```python
import pandas as pd
import re

df = pd.read_csv("lyrics/lyrics_full.csv", sep=',', encoding='utf-8')  # Fichier non disponbile
df = df[~df["lyrics"].isna()]  # On enlève les NaNs
text = df[df["genre"]=='Hip-Hop']  # Filtre Hip-Hop
text = text[text.lyrics.apply(len) > 200]  # On garde les paroles avec assez de texte
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\?\s*\n+", '? ', x))  # "? \n"  ->   "? "
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\!\s*\n+", '! ', x))  # "! \n"  ->   "! "
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\.\s*\n+", '. ', x))  # ". \n"  ->   ". "
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\n+", ' ', x))  # "\n\n"  ->   " "
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\!+\.+", '!', x))  # "!."  ->   "!"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\?+\.+", '?', x))  # "?."  ->   "?"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\[[\w\d\s,:\(\)\_\-\+\?\!\.\"_;/\\\*\[]*\]", ' ', x))  # "(text)"  ->   ""
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\([\w\d\s,:\[\]\_\-\+\?\!\.\"_;/\\\*\(]*\)", ' ', x))  # "[text]"  ->   ""
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\!(?=\w)", '! ', x))  # "!text"  ->   "! text"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\?(?=\w)", '? ', x))  # "?text"  ->   "? text"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\.(?=\w)", '. ', x))  # ".text"  ->   ". text"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"(?<=\w)\!", ' !', x))  # "text!"  ->   "text !"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"(?<=\w)\?", ' ?', x))  # "text?"  ->   "text ?"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"(?<=\w)\.", ' .', x))  # "text."  ->   "text ."
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\.{2,}", '.', x))  # "..."  ->   "."  
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\!{2,}", '!', x))  # "!!!"  ->   "!"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"\?{2,}", '?', x))  # "???"  ->   "?"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"[,:\[\]\(\)\"_\-\+_;/\\\*]", ' ', x))  # "text, -zea"  ->   "text zea"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r'[^\x00-\x7f]', '', x))  # Suppression de certains caractères non ascii
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"[\t\f\v ]{2,}", ' ', x))  # "text  zea"  ->   "text zea"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"^(\s)+", '', x))  # " text zea"  ->   "text zea"
text["lyrics"] = text["lyrics"].apply(lambda x: re.sub(r"(\s)+$", '', x))  # "text zea "  ->   "text zea"

text[['song', 'artist', 'year', 'lyrics']].reset_index(drop=True).to_csv('lyrics/lyrics_hip_hop.csv', sep=';', encoding='utf-8', index=None)
```